# Pytorch를 이용한 간단한 CNN 구현

## 1. 필요한 라이브러리 불러오기

In [12]:
import torch    # Pytorch 기본 라이브러리
import torch.nn as nn   # Neural Network (nn)에 관련된 모듈들
from torch.utils.data import DataLoader # Database의 데이터들을 불러오기 위한 라이브러리

from torchvision import transforms, datasets    # 불러온 입력을 처리하는 모듈들

import os   # 경로 탐색, 접근 등을 처리하기 위한 라이브러리
import numpy as np # 행렬 연산에 사용하는 라이브러리

## 2. 평가를 위한 하이퍼 파리미터 설정

In [13]:
batch_size = 128

ckpt_dir = './checkpoint'   # 학습된 파라미터를 저장할 경로

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
print(device)

cpu


## 3. CNN 구축

In [16]:
class Net(nn.Module):       # nn.Module를 상속받아서 사용.
    def __init__(self):
        super(Net, self).__init__() # 상속해준 부모 클래스의 Method를 사용할 때 super(하위클래스, 하위클래스의 객체)

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=10, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)    # cf. AvgPool2d

        # Block 2
        self.conv2 = nn.Conv2d(in_channels=10, 
                               out_channels=20, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)    # H * W * C = 320

        self.fc1 = nn.Linear(in_features=320, 
                             out_features=50)             # Fully-Connected Layer
        self.relu1_fc1 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):   # x: 입력영상 (torch.Tensor())
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(-1, 320)     # x: B=128, C=20, H=4, W=4

        x = self.fc1(x)
        x = self.relu1_fc1(x)

        x = self.fc2(x)

        return x

## 4. 학습된 파라미터를 저장하거나 불러오는 함수 작성

In [17]:
def save(ckpt_dir, net, optim, epoch):      # ckpt_dir: checkpoint를 저장할 경로, net, optim, epoch
    if not os.path.exists(ckpt_dir):    # ckpt_dir이 존재하는지 확인하는 함수
        os.makedirs(ckpt_dir)           # 디렉토리를 만들어주는 함수

    torch.save({'net': net.state_dict(),        # 네트워크에 있는 변수들
                'optim': optim.state_dict()},   # optimizer에 있는 변수들
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim=None):
    ckpt_lst = os.listdir(ckpt_dir)  # 입력한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 반환
    ckpt_lst.sort()  # 정렬

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    
    if optim is not None:
        optim.load_state_dict(dict_model['optim'])

    return net, optim

## 5. Database 불러오기
### Database가 없을 경우 Pytorch에 내장되어 있는 MNIST 다운로드 함수 사용

In [18]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, 
                         root='./', 
                         train=False, 
                         transform=transform)   # 개별 데이터 파일의 처리를 담당
loader = DataLoader(dataset, 
                    batch_size=batch_size, 
                    shuffle=False, 
                    num_workers=0)  # 미니배치 처리를 담당

num_data = len(loader.dataset)
num_batch = num_data // batch_size

## 6. 네트워크 및 평가함수 선언

In [21]:
net = Net()
net = net.to(device)

fn_loss = nn.CrossEntropyLoss().to(device)

def fn_pred(output):
    return torch.softmax(output, dim=1)     # B=128 (dim=0), 10 (dim=1)

def fn_accuracy(pred, label):
    return (pred.max(dim=1)[1] == label).type(torch.float).mean()

## 7. 성능 검증

In [22]:
num_data = len(loader.dataset)
num_batch = num_data // batch_size

net, _ = load(ckpt_dir=ckpt_dir, net=net)

## 평가 시작하기
with torch.no_grad():
    # net.train()
    net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader, 1):
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)

        loss = fn_loss(output, label)
        acc = fn_accuracy(pred, label)

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TEST: BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

TEST: BATCH 0001/0078 | LOSS: 0.0515 | ACC 0.9922
TEST: BATCH 0002/0078 | LOSS: 0.0599 | ACC 0.9883
TEST: BATCH 0003/0078 | LOSS: 0.0936 | ACC 0.9740
TEST: BATCH 0004/0078 | LOSS: 0.0993 | ACC 0.9727
TEST: BATCH 0005/0078 | LOSS: 0.1035 | ACC 0.9688
TEST: BATCH 0006/0078 | LOSS: 0.1104 | ACC 0.9648
TEST: BATCH 0007/0078 | LOSS: 0.1063 | ACC 0.9676
TEST: BATCH 0008/0078 | LOSS: 0.1157 | ACC 0.9629
TEST: BATCH 0009/0078 | LOSS: 0.1204 | ACC 0.9583
TEST: BATCH 0010/0078 | LOSS: 0.1378 | ACC 0.9555
TEST: BATCH 0011/0078 | LOSS: 0.1426 | ACC 0.9545
TEST: BATCH 0012/0078 | LOSS: 0.1466 | ACC 0.9531
TEST: BATCH 0013/0078 | LOSS: 0.1445 | ACC 0.9543
TEST: BATCH 0014/0078 | LOSS: 0.1494 | ACC 0.9542
TEST: BATCH 0015/0078 | LOSS: 0.1460 | ACC 0.9557
TEST: BATCH 0016/0078 | LOSS: 0.1498 | ACC 0.9551
TEST: BATCH 0017/0078 | LOSS: 0.1516 | ACC 0.9540
TEST: BATCH 0018/0078 | LOSS: 0.1577 | ACC 0.9531
TEST: BATCH 0019/0078 | LOSS: 0.1587 | ACC 0.9531
TEST: BATCH 0020/0078 | LOSS: 0.1549 | ACC 0.9543
